# Semi-Greedy Construction

Before we can implement a full GRASP for the TSP we need to understand semi-greedy construction. 

---
> **Greedy construction**: we select a starting point in the tour and then choose the next closest citiy to travel to.  We continue to do this until all cities are visited.
>> **Semi-Greedy construction**: we now incorporate probability in the greedy construction.  Instead of always visiting the closest city there is now a probability that we visit one of the **8** closest cities.

So you can think of **semi-greedy** algorithms as a half way house between random and greedy search.

---

## Imports


In [1]:
import numpy as np

## `metapy` imports

In [ ]:
# uncomment if you are running in Google colab
# !pip install meta-py

In [2]:
from metapy.tsp import tsp_io as io
from metapy.tsp.euclidean import gen_matrix, plot_tour

## Restricted Candidate Lists

A key ingredient in semi-greedy construction is the **restricted candidate list**: the list of top valued components that the algorithm will choose from in its next step.  In the TSP a straightfoward implementation is choosing the closest **r** cities where r is a hyperparameter that can be fixed, a random variable or **learned** over time.

Here we introduce two classes: `FixedRCLSizer` and `RandomRCLSizer` that provide a constant and probabilitically selected size of the RCL, repectively

In [3]:
class FixedRCLSizer:
    '''
    Fixed sized RCL list.
    
    When r = 1 then greedy
    When r = len(tour) then random
    '''
    def __init__(self, r):
        self.r = r
        
    def get_size(self):
        '''
        Returns an int representing the size of the required RCL
        '''
        return self.r

In [4]:
class RandomRCLSizer:
    '''
    Probabilitic selection of the RCL size
    Uniform probability.
    '''
    def __init__(self, r_list, random_seed=None):
        self.r_list = r_list
        self.rng = np.random.default_rng(random_seed)
        
    def get_size(self, size=None):
        '''
        Returns a randomly selected RCL size
        '''
        return self.rng.choice(self.r_list, size=size)

## Semi-greedy implementation

We are now ready to build a basic semi-greedy implementation.

> A key point to remember is that construction is **iterative**. A new RCL is created in each iteration and a city is chosen from it at random until all cities have been added to the tour.

Here we create a `SemiGreedyConstructor` class.  The class accepts a **rcl_sizer** as an argument (an object that implements a `get_size()` method).  This means you can vary the RCL sizing logic and treat it as a hyper parameter.

In [5]:
class SemiGreedyConstructor:
    '''
    Semi-greedy construction of a tour.
    
    For a city i creates a restricted candidate list of size r
    i.e the r shortest distances from city i.  
    Next city is chosen with equal probability.
    Repeats until tour is constructed.
    '''
    def __init__(self, rcl_sizer, tour, matrix,
                 random_seed=None):
        '''
        Constructor
        
        Params:
        ------
        rcl_sizer: object
            sizes the restricted candidate list
        
        tour: np.ndarray
            vector of city indexes included in problem
            
        matrix: np.ndarray
            matrix of travel costs
            
        random_seed: int
            used to control sampling and provides a
            reproducible result.
        '''
        
        # size of rcl
        self.rcl_sizer = rcl_sizer
        
        # cities in a tour
        self.tour = tour
        
        # travel cost matrix
        self.matrix = matrix
        
        # create random number generator
        self.rng = np.random.default_rng(random_seed)
    
    def build(self):
        '''
        Semi-greedy contruction of tour
        
        Returns:
        --------
        np.array
        '''
        # first city in tour
        solution = np.array([self.tour[0]])    
        
        # it is an iterative (construction) procedure
        for i in range(len(self.tour)-1):
            # get the RCL size
            r = self.rcl_sizer.get_size()
            
            # get the RCL
            rcl = self.get_rcl(r, solution, solution[-1])
            
            # select the next city 
            next_city = self.random_from_rcl(rcl)
            
            # update the solution
            solution = np.append(solution, np.array([next_city]))
            
        return solution
    
    def get_rcl(self, r, solution, from_city):
        '''
        Restricted candidate list for final city in current solution
        
        Params:
        -------
        solution: np.ndarray
            vector of current partially constructed solution
            
        from_city: int
            index of city used to construct rcl.
        
        Returns:
        -------
        np.array
        '''
        # get indexes of cities not in solution
        mask = self.tour[~np.in1d(self.tour, solution)]
        
        # get indexes of r smallest travels costs 
        if mask.shape[0] > r:
            # partition the vector for remaining cities - faster than sorting 
            idx = np.argpartition(self.matrix[from_city][mask], 
                                  len(mask) - r)[-r:]
            rcl = mask[idx]
        else:
            # handle when r < n cities remaining 
            rcl = mask
        return rcl
    
    def random_from_rcl(self, rcl):
        '''
        Select a city at random from rcl.
        Return city index in self.matrix
        
        Params:
        -------
        rcl: np.ndarray
            restricted candidate list
            vector of candidate city indexes.
        '''
        return self.rng.choice(rcl)

In [6]:
#load file
file_path = 'https://raw.githubusercontent.com/TomMonks/meta-py/main/data/st70.tsp'

#number of rows in the file that are meta_data
md_rows = 6

#read the coordinates
cities = io.read_coordinates(file_path, md_rows)
matrix = gen_matrix(cities, as_integer=True)

In [7]:
# when r = 1 then greedy.  
# when r = len(tour) then random solution

tour = np.arange(70)
sizer = FixedRCLSizer(r=2)
constructor = SemiGreedyConstructor(sizer, tour, -matrix)
constructor.build()

array([ 0, 15, 46, 57, 36, 49,  9, 51,  4, 59, 11, 20, 33, 16, 40,  5, 41,
       17,  3,  1,  6, 18, 25,  7, 27,  2, 13, 19, 43, 67, 29, 26, 39, 60,
       44, 24, 45, 38,  8, 61, 32, 53, 66, 55, 64, 50, 47, 63, 10, 52, 21,
       65, 62, 58, 37, 30, 68, 69, 28, 35, 12, 34, 56, 23, 14, 22, 31, 42,
       54, 48])

In [8]:
# alternative setup with random sizer
# pass RandomRCLSizer a list or np.array of ints of r.
sizer = RandomRCLSizer(np.arange(1, 15))
constructor = SemiGreedyConstructor(sizer, tour, -matrix)
constructor.build()

array([ 0, 35, 15, 46, 37, 28, 30, 62, 21, 23,  1, 18, 25, 27,  7, 13, 67,
       42, 40, 31, 43, 29, 17,  3, 58, 68, 14,  6, 48, 54, 16,  5,  9, 59,
       10, 55, 49, 11, 33, 61, 53, 64, 50,  4, 52, 51,  8, 45, 41, 26, 38,
       24,  2, 20, 57, 22, 36, 65, 56, 12, 32, 69, 44, 39, 34, 66, 63, 19,
       47, 60])